## CRSP

### tbtf database and crsp_raw

In [ ]:
import pandas as pd
import sqlite3
tidy_finance = sqlite3.connect(database="../../tidy_finance_python.sqlite")

cursor = tidy_finance.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# 1994-01-01 indicates mktcap at 1994-01-31 which is the start date
# Thus the very first return is calculated from 1994-02 which means 1994-02-28 in 'date' field or 1994-02-01 in 'month' field
# industry 는 siccd 를 이용해서 분류함
# mktcap_lag 는 mktcap을 x["month"]+pd.DateOffset(months=1) 을 이용해서 할당
# ret_excess 은 FF data library에서 rf를 불러와서 계산

crsp_monthly = pd.read_sql_query(
  sql="SELECT * FROM crsp_monthly",
  con=tidy_finance,
  parse_dates={"date"}
)

crsp_monthly.head()

[('factors_ff3_monthly',), ('factors_ff5_monthly',), ('factors_ff3_daily',), ('industries_ff_monthly',), ('factors_q_monthly',), ('macro_predictors',), ('cpi_monthly',), ('crsp_daily',), ('compustat',), ('beta',), ('crsp_monthly',)]


permno       date                month       ret   shrout  altprc  \
0   10001 1994-02-28  1994-02-01 00:00:00  0.000000  1091000  17.500   
1   10001 1994-03-31  1994-03-01 00:00:00 -0.004429  1091000  17.250   
2   10001 1994-04-29  1994-04-01 00:00:00 -0.144928  1091000  14.750   
3   10001 1994-05-31  1994-05-01 00:00:00  0.067797  1091000  15.750   
4   10001 1994-06-30  1994-06-01 00:00:00  0.108044  2191000   8.625   

  primaryexch  siccd     mktcap  mktcap_lag exchange   industry  ret_excess  
0           Q   4925  19.092500    19.09250   NASDAQ  Utilities   -0.002100  
1           Q   4925  18.819750    19.09250   NASDAQ  Utilities   -0.007129  
2           Q   4925  16.092250    18.81975   NASDAQ  Utilities   -0.147628  
3           Q   4925  17.183250    16.09225   NASDAQ  Utilities    0.064697  
4           Q   4925  18.897375    17.18325   NASDAQ  Utilities    0.104944

In [4]:
crsp_raw = pd.read_sql_query(
  sql="SELECT permno, date, mktcap, ret, primaryexch, siccd, industry, mktcap_lag, ret_excess FROM crsp_monthly",
  con=tidy_finance,
  parse_dates={"date"}
)

start_date = '1996-01-01'
end_date = '2023-12-31'

crsp_raw = crsp_raw[(crsp_raw['date'] >= start_date) & (crsp_raw['date'] <= end_date)]

crsp_raw.head()

permno       date     mktcap       ret primaryexch  siccd   industry  \
23   10001 1996-01-31  20.814125 -0.026667           Q   4925  Utilities   
24   10001 1996-02-29  21.099250  0.013699           Q   4925  Utilities   
25   10001 1996-03-29  21.899480  0.036423           Q   4925  Utilities   
26   10001 1996-04-30  20.348063 -0.070840           Q   4925  Utilities   
27   10001 1996-05-31  19.915125 -0.021277           Q   4925  Utilities   

    mktcap_lag  ret_excess  
23   21.384375   -0.030967  
24   20.814125    0.009799  
25   21.099250    0.032523  
26   21.899480   -0.075440  
27   20.348063   -0.025477

In [5]:
print(crsp_raw['primaryexch'].unique())

['Q' 'A' 'N' 'X' 'B' 'R' 'I']


각 거래소 코드의 의미 (각 주식이 어느 거래소에서 주로 거래되는지를 식별)

1. 'Q': NASDAQ (National Association of Securities Dealers Automated Quotations)
   - 미국의 대표적인 기술주 중심 전자 증권 거래소
   - Apple, Microsoft, Google 등 많은 테크 기업들이 상장되어 있음

2. 'A': NYSE American (구 American Stock Exchange, AMEX)
   - 중소기업과 신흥 기업들이 주로 상장되는 거래소
   - 소형 및 중형 기업들에게 상장 기회를 제공

3. 'N': NYSE (New York Stock Exchange)
   - 세계에서 가장 큰 규모의 증권 거래소
   - 대형 우량 기업들이 주로 상장됨
   - 전통적이고 권위 있는 거래소로 알려져 있음

4. 'X': NASDAQ OMX PHLX (Philadelphia Stock Exchange)
   - 옵션 거래에 특화된 거래소
   - 현재는 NASDAQ의 일부로 운영됨

5. 'B': NASDAQ BX (NASDAQ OMX Boston)
   - NASDAQ의 일부인 지역 거래소
   - 주식 거래에 특화됨

6. 'R': Regional Exchanges (지역 증권 거래소)
   - 미국의 다양한 지역 증권 거래소를 포괄하는 코드
   - 주요 거래소 외의 지역 거래소들을 나타냄

7. 'I': Other International Exchanges (기타 국제 거래소)
   - 미국 외 국제 증권 거래소들을 포함하는 코드
   - 글로벌 거래소들을 대표

In [7]:
import sqlite3
# SQLite DB 파일 생성
# con = sqlite3.connect("tbtf.sqlite")
# database 연결
con = sqlite3.connect(database="../../tbtf.sqlite")

# database에 dataframe 넣기
crsp_raw.to_sql(name="crsp_raw", con=con, if_exists="replace", index=False)

# 최종 DB 저장이 완료된 후, 배포·보관 목적의 최종 정리 단계에서, DB 파일 단위의 물리적 최적화
# tbtf.execute("VACUUM")

1584445

### crsp: crsp_raw에 state & state_lag 생성 

left tail truncation 해결.

In [24]:
# 새로운 database를 연결해서 저장해 놓은 crsp dataframe을 확인

crsp_raw = pd.read_sql_query(
  sql="SELECT * FROM crsp_raw",
  con=con,
  parse_dates={"date"}
)
crsp_raw.head()


permno       date     mktcap       ret primaryexch  siccd   industry  \
0   10001 1996-01-31  20.814125 -0.026667           Q   4925  Utilities   
1   10001 1996-02-29  21.099250  0.013699           Q   4925  Utilities   
2   10001 1996-03-29  21.899480  0.036423           Q   4925  Utilities   
3   10001 1996-04-30  20.348063 -0.070840           Q   4925  Utilities   
4   10001 1996-05-31  19.915125 -0.021277           Q   4925  Utilities   

   mktcap_lag  ret_excess  
0   21.384375   -0.030967  
1   20.814125    0.009799  
2   21.099250    0.032523  
3   21.899480   -0.075440  
4   20.348063   -0.025477

In [ ]:
from pandas.tseries.offsets import MonthEnd

# 날짜 정렬 및 월말 정렬 (CRSP 구조 특화)
# 월별 구조에서 흔히 발생하는 alignment error 피해가기
# crsp 전체 date를 월말로 정렬 (이 작업은 최초 데이터 생성 시 1회만 하면 됨)
crsp_raw['date'] = pd.to_datetime(crsp_raw['date']) + MonthEnd(0)
crsp_raw = crsp_raw.sort_values(['date', 'permno']).copy()

# mktcap이 존재하는 날짜에 대해 state 계산
crsp_valid = crsp_raw.dropna(subset=['mktcap']).copy()

# assign_state: mktcap 기준 decile (1~10)
def assign_state(group):
    group = group.copy()
    group['state'] = pd.qcut(
        group['mktcap'],
        q=10,
        labels=False, # 0~9 범위
        duplicates='drop'
    ) + 1 # → 1~10 범위로 변환
    return group

# 날짜별로 시가총액 기준 qcut 적용
# state 생성: permno, date, state 만 따로 추출
state_df = (
    crsp_valid
    .set_index(['permno', 'date'])  # 그룹핑 컬럼을 인덱스로 설정
    .groupby(level='date', group_keys=False)
    .apply(assign_state)
    .reset_index()[['permno', 'date', 'state']]
)

# 원래 데이터에 병합
crsp = crsp_raw.merge(state_df, on=['permno', 'date'], how='left')

# 정확한 state_lag 계산 → permno별 시계열 기준으로 전월 값 shift
crsp = crsp.sort_values(['permno', 'date']).copy()
crsp['state_lag'] = crsp.groupby('permno')['state'].shift(1)

# 결측값 (예: 신규 상장 종목, 상장 폐지 등)은 0으로 처리
# state_df를 crsp_raw에 병합했을 때, state는 일부 permno–date 조합에서 누락됨(NaN).
crsp['state'] = crsp['state'].fillna(0).astype(int) 
crsp['state_lag'] = crsp['state_lag'].fillna(0).astype(int)

# 정렬
crsp = crsp.sort_values(['permno', 'date']).reset_index(drop=True)

crsp.head(10)

permno       date     mktcap       ret primaryexch  siccd   industry  \
0   10001 1996-01-31  20.814125 -0.026667           Q   4925  Utilities   
1   10001 1996-02-29  21.099250  0.013699           Q   4925  Utilities   
2   10001 1996-03-31  21.899480  0.036423           Q   4925  Utilities   
3   10001 1996-04-30  20.348063 -0.070840           Q   4925  Utilities   
4   10001 1996-05-31  19.915125 -0.021277           Q   4925  Utilities   
5   10001 1996-06-30  18.568000 -0.061490           Q   4925  Utilities   
6   10001 1996-07-31  19.003187  0.023438           Q   4925  Utilities   
7   10001 1996-08-31  19.728500  0.038168           Q   4925  Utilities   
8   10001 1996-09-30  20.527500  0.041944           Q   4925  Utilities   
9   10001 1996-10-31  19.941000 -0.028571           Q   4925  Utilities   

   mktcap_lag  ret_excess  state  state_lag  
0   21.384375   -0.030967      3          0  
1   20.814125    0.009799      3          3  
2   21.099250    0.032523      3          3  
3   21.899480   -0.075440      2          3  
4   20.348063   -0.025477      2          2  
5   19.915125   -0.065490      2          2  
6   18.568000    0.018938      2          2  
7   19.003187    0.034068      2          2  
8   19.728500    0.037544      2          2  
9   20.527500   -0.032771      2          2

In [ ]:
# crsp 에 state을 추가해서는 샘플기간 내에 상장폐지 개수를 확인할 수 없음.
# 따라서 중간 상장폐지 기업을 고려하여 transition matrix 계산을 위해서는 
# 전처리 과정에서 임의로 state=0을 부여함.
crsp[(crsp['state'] == 0) & (crsp['date'] > "1996-01-31")].shape[0]

0

In [ ]:
# 샘플기간 내에 신규상장 개수
crsp[(crsp['state_lag'] == 0) & (crsp['state'] > 0) & (crsp['date'] > '1996-01-31')].shape[0]

8870

In [31]:
# database에 dataframe 넣기
crsp.to_sql(name="crsp", con=con, if_exists="replace", index=False)

1584445

## Fama French

Field 설명 
- date: monthly frequency
- ret of porftolio
좋아. 내가 의도한 것은 각 breakpoints 구간 별로 ME (in millions) per a Firm 를 구하는 거였어. 아래 코드를 깔끔하게 부탁해.

### Mkt and RF

In [3]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
pd.set_option('display.width', None)
pd.set_option('display.max_columns', 15)

start_date = '1996-01-01'
end_date = '2023-12-31'

ff3_raw = pdr.DataReader(
  name="F-F_Research_Data_Factors",
  data_source="famafrench",
  start=start_date,
  end=end_date)

ff3 = (ff3_raw[0]
  .divide(100)
  .reset_index()
  .assign(date = lambda x: pd.to_datetime(x["Date"].astype(str))) # change from 01 to 01-01
  .assign(date = lambda x: x['date'].dt.to_period('M').dt.to_timestamp('M')) # change from 01-01 to 01-31
  .assign(mkt_ret = lambda x: x['Mkt-RF']+x['RF'])
  .rename(columns = {'RF': 'r_f'})
  .get(['date', 'mkt_ret', 'r_f'])
)
ff3.head().round(3)

date  mkt_ret    r_f
0 1996-01-31    0.027  0.004
1 1996-02-29    0.017  0.004
2 1996-03-31    0.011  0.004
3 1996-04-30    0.025  0.005
4 1996-05-31    0.028  0.004

In [5]:
import sqlite3
tbtf = sqlite3.connect("tbtf.sqlite")
ff3.to_sql(name="ff3", con=tbtf, if_exists="replace", index=False)

336

### Fama-French ME × PRIOR 
- The portfolios, which are constructed monthly, are the intersections of 5 portfolios formed on size (market equity, ME) and 5 portfolios formed on prior (2-12) return.
- Focus on the persistence of high prior year return momentum, across different market cap groups. 


지난 1달을 제외함으로써 단기 변동성을 줄이고 더 안정적인 모멘텀 신호를 얻으려는 의도

1. "Prior (2-12) return"는 지난 1달(t-1)을 건너뛰고 그전 11개월(t-2부터 t-12까지)의 수익률을 매달 계산하는 롤링(rolling) 방식으로 사용됩니다. 이 수익률 데이터를 기반으로 NYSE 주식들의 분포에서 30번째와 70번째 백분위수를 찾아 분기점(breakpoint)을 설정한다는 뜻입니다. 즉, 매달 새로운 한 달 치 데이터가 추가되고 가장 오래된 한 달 치가 제외되면서 계속 업데이트되는 방식입니다. 예: 2025년 3월(t=0)이라면, 2025년 1월(t-2)부터 2024년 2월(t-12)까지의 11개월 데이터를 사용합니다. 다음 달(2025년 4월)이 되면 2025년 2월(t-2)부터 2024년 3월(t-12)까지로 이동합니다.

2. **왜 지난 1달은 제외했나?**: 지난 1달(t-1)을 넣지 않는 이유는 주로 **리버설 효과(reversal effect)**나 **단기 노이즈(short-term noise)**를 피하기 위한 것으로 보입니다. 금융 연구에서 모멘텀 전략을 설계할 때, 직전 1개월 수익률을 포함하면 단기적인 가격 반전(예: 과매수나 과매도 후 반대 방향으로 움직이는 경향)이 모멘텀 신호를 왜곡할 수 있습니다. 반면, t-2부터 t-12까지의 11개월 데이터를 사용하면 중기적인 추세(mid-term trend)를 더 잘 포착할 수 있다고 여겨집니다. 이는 Fama-French 같은 학자들이 모멘텀 효과를 분석할 때 자주 사용하는 관행이기도 합니다. 또한, 실무적으로 포트폴리오를 구성하는 시점에서 직전 달 데이터가 완전히 확정되지 않았을 가능성도 고려될 수 있습니다.


In [ ]:
start_date = '2010-01-01'
end_date = '2023-12-31'

import numpy as np
import pandas as pd
import pandas_datareader as pdr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
pd.set_option('display.width', None)
pd.set_option('display.max_columns', 15)

ff = pdr.DataReader(
  name="25_Portfolios_ME_Prior_12_2",
  data_source="famafrench",
  start=start_date,
  end=end_date)

# print(ff['DESCR'])
# ff[0].columns.tolist()

# Value-weighted is 0, while Equal-weighted is 1
ff_vw = (ff[0]
  .rename(columns = {'BIG HiPRIOR':'ff_b', 'ME3 PRIOR5': 'ff_m', 'SMALL HiPRIOR': 'ff_s'})
  .divide(100)
  .reset_index()
  .assign(date = lambda x: pd.to_datetime(x["Date"].astype(str))) # change from 01 to 01-01
  .assign(date = lambda x: x['date'].dt.to_period('M').dt.to_timestamp('M')) # change from 01-01 to 01-31
  .get(['date', 'ff_b', 'ff_m', 'ff_s'])
)

ff_ew = (ff[1]
  .rename(columns = {'BIG HiPRIOR':'ff_e_b', 'ME3 PRIOR5': 'ff_e_m', 'SMALL HiPRIOR': 'ff_e_s'})
  .divide(100)
  .reset_index()
  .assign(date = lambda x: pd.to_datetime(x["Date"].astype(str))) # change from 01 to 01-01
  .assign(date = lambda x: x['date'].dt.to_period('M').dt.to_timestamp('M')) # change from 01-01 to 01-31
  .get(['date', 'ff_e_b', 'ff_e_m', 'ff_e_s'])
)

ff = ff_vw.merge(ff_ew, how = 'inner', on='date')
ff = ff[['date', 'ff_b', 'ff_m', 'ff_s', 'ff_e_b', 'ff_e_m', 'ff_e_s']] # 컬럼 순서 정리

ff.head()

date    ff_b    ff_m    ff_s  ff_e_b  ff_e_m  ff_e_s
0 2010-01-31 -0.0877 -0.0522 -0.0352 -0.0778 -0.0547 -0.0014
1 2010-02-28  0.0589  0.0756  0.0816  0.0569  0.0723  0.0715
2 2010-03-31  0.0969  0.0877  0.0922  0.1233  0.0914  0.0983
3 2010-04-30  0.0309  0.0465  0.1255  0.0389  0.0477  0.1245
4 2010-05-31 -0.0776 -0.0996 -0.0889 -0.0814 -0.0971 -0.0957

In [31]:
import sqlite3

# 데이터베이스 연결
tbtf = sqlite3.connect("tbtf.sqlite")
cursor = tbtf.cursor()

# 테이블 이름 변경
cursor.execute("ALTER TABLE ff RENAME TO post_ff;")
tbtf.commit()

## Yahoo Finance

### Post-2010 ETFs
'VTI', 'SPY', 'QQQ', 'DIA'

In [6]:
start_date = '2009-12-20'
end_date = '2023-12-31'

import numpy as np
import pandas as pd
import yfinance as yf

index_prices_daily = (yf.download(
    tickers=['VTI', 'SPY', 'QQQ', 'DIA'], # symbol list
    start=start_date,
    end=end_date,
    auto_adjust=False,  # 명시적으로 auto_adjust=False 설정
    progress=False
  )
  .stack(level=1, future_stack=True)
  .reset_index()
  .rename(columns={
    "Date": "date",
    "Ticker": 'ticker',
    "Close": "price"}
  )
  .assign(price = lambda x: x["price"] / x.groupby("ticker")["price"].transform('first'))
  .get(["date", "ticker", "price"])
)

index_prices_monthly_raw = (
    index_prices_daily
    .pivot(index="date", columns="ticker", values="price")
    .resample("ME")
    .last()
    .dropna()
)

# Normalize using the *first monthly close*, not the first daily close
base_prices = index_prices_monthly_raw.iloc[0]
index_prices_monthly = index_prices_monthly_raw.divide(base_prices)

index_prices_monthly = (
    index_prices_monthly
    .reset_index()
    .assign(date=lambda x: x['date'].dt.tz_localize(None))
    .set_index('date')
)

index_returns_monthly = (index_prices_monthly
                         .pct_change()
                         .dropna()
                         .reset_index()
)

# Wide to long format for DB compatibility
#index_returns_monthly_long = (
#    index_returns_monthly.melt(id_vars='date', var_name='ticker', value_name='ret')  # tidy format
#)

# Save to SQLite
import sqlite3
tbtf = sqlite3.connect("tbtf.sqlite")

# post-2010 ETF returns
index_returns_monthly.to_sql(name="post_index", con=tbtf, if_exists="replace", index=False)

168

### Pre-2010 Indices
- ^NDX (Nasdaq 100 Index)
- ^DJI (Dow Jones Industrial Average)

In [7]:
start_date = '1995-12-20'
end_date = '2009-12-31'

import numpy as np
import pandas as pd
import yfinance as yf

index_prices_daily = (yf.download(
    tickers=['^NDX', '^DJI'], # symbol list
    start=start_date,
    end=end_date,
    auto_adjust=False,  # 명시적으로 auto_adjust=False 설정
    progress=False
  )
  .stack(level=1, future_stack=True)
  .reset_index()
  .rename(columns={
    "Date": "date",
    "Ticker": 'ticker',
    "Close": "price"}
  )
  .assign(price = lambda x: x["price"] / x.groupby("ticker")["price"].transform('first'))
  .get(["date", "ticker", "price"])
)

index_prices_monthly_raw = (
    index_prices_daily
    .pivot(index="date", columns="ticker", values="price")
    .resample("ME")
    .last()
    .dropna()
)

# Normalize using the *first monthly close*, not the first daily close
base_prices = index_prices_monthly_raw.iloc[0]
index_prices_monthly = index_prices_monthly_raw.divide(base_prices)

index_prices_monthly = (
    index_prices_monthly
    .reset_index()
    .assign(date=lambda x: x['date'].dt.tz_localize(None))
    .set_index('date')
)

index_returns_monthly = (index_prices_monthly
                         .pct_change()
                         .dropna()
                         .reset_index()
)

# Wide to long format for DB compatibility
#index_returns_monthly_long = (
#    index_returns_monthly.melt(id_vars='date', var_name='ticker', value_name='ret')  # tidy format
#)

# Save to SQLite
import sqlite3
tbtf = sqlite3.connect("tbtf.sqlite")

# post-2010 ETF returns
index_returns_monthly.to_sql(name="pre_index", con=tbtf, if_exists="replace", index=False)

168

In [19]:
cursor = tbtf.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('crsp',), ('post_ff',), ('ff3',), ('post_index',), ('pre_index',)]
